In [1]:

import numpy as np
import pandas as pd
import ROOT as r
import os 
import sys
import time
import uproot 
from functools import partial
from triggerConstants import *

Welcome to JupyROOT 6.24/07


/cvmfs/cms.cern.ch/slc7_amd64_gcc10/external/py3-setuptools/60.9.3-ddcab3b6a742c72fed5554de8ccaaf18/lib/python3.9/site-packages/_distutils_hack/__init__.py:17: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/cvmfs/cms.cern.ch/slc7_amd64_gcc10/external/py3-setuptools/60.9.3-ddcab3b6a742c72fed5554de8ccaaf18/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [2]:
class triggerChecker():

	def __init__(self):
		self.defineFunctions()

	#check for nHits pulses in window
	#Q: what does (x.unique() < 64) do?
	def checkNPulses(self, x, nHits=4):
		if len((x.unique()<64)) >= nHits: return True
		return False
	
	#check for NLayers hits in window
	def checkNLayers(self, x, nLayers=4):
		if len(x) < nLayers: return False
		unique = x.unique()
		layers = np.where((unique >= 0) & (unique < 4))
		return layers[0].size >= nLayers

	#return value of column (for debugging)
	def returnValue(self, x):
		return x.values[0]

	#check for separate layers in window
	def checkSeparate(self, x):
		hit = (np.array([np.in1d(combo, x) for combo in separateCombos]).all(1)).any()
		return hit

	#check for adjacent layers in window
	def checkAdjacent(self, x):
		hit = (np.array([np.in1d(combo, x) for combo in adjacentCombos]).all(1)).any()
		return hit
	
	#check for top panels trigger in window
	def checkTopPanels(self, x, bot=False):
		hit = np.array([np.in1d(panel, x) for panel in topPanels]).any()
		if not bot: return hit    
		extra = np.array([np.in1d(chan, x) for chan in bottomBars]).any()
		return extra*hit
	
	#check for front+back panels in window
	def checkThroughGoing(self, x):
		hit = np.array([np.in1d(panel, x) for panel in frontBack]).all()
		return hit
		
	#check for 3 in row trigger in window
	def checkThreeInRow(self, x):
		passed = np.array([np.count_nonzero([np.in1d(path, x) for path in line]) >= 3 for line in straightPaths]).any()
		return passed

	#define columns replaced by trigger finding
	def defineTrigCols(self):
		self.myarray['fourLayers'] = self.myarray['layer']
		self.myarray['threeInRow'] = self.myarray['chan']
		self.myarray['separateLayers'] = self.myarray['layer']
		self.myarray['adjacentLayers'] = self.myarray['layer']
		self.myarray['nLayers'] = self.myarray['layer']
		self.myarray['nHits'] = self.myarray['layer']
		self.myarray['topPanels'] = self.myarray['chan']
		self.myarray['topBotPanels'] = self.myarray['chan']
		self.myarray['panelsCleaned'] = np.where((self.myarray['height'] >= panelThreshold), self.myarray['chan'], False)
		self.myarray['frontBack'] = self.myarray['panelsCleaned']

	#function to open the input root file
	def openFile(self, dataIn):
		self.input_file = dataIn
		self.runNum = int(dataIn.split('Run')[-1].split('.')[0])
		self.fileNum = int(dataIn.split('.')[1].split('_')[0])
		fin = uproot.open(dataIn)
		tree = fin['t']
		self.myarray = tree.arrays(uprootInputs, library='pd')
		#self.myarray = tree.arrays(uprootEventBasedInput, library='pd')
		#self.setTimes()
		#self.defineTrigCols()

	#use datetime to create times of hits to be used in rolling window
	def setTimes(self):
		self.myarray['fake_time'] = pd.to_datetime((self.myarray['time']+self.myarray['event']*86400), unit='s')  
		self.myarray = self.myarray.sort_values(by=['entry', 'time'], ascending=True)

	#define specific trigger functions from generic functions
	def defineFunctions(self):
		self.thisNLayers = partial(self.checkNLayers, nLayers=nLayers)
		self.thisNPulses = partial(self.checkNPulses, nHits=nHits)
		self.topBotPanels = partial(self.checkTopPanels, bot=True)
		self.this4Layers = partial(self.checkNLayers, nLayers=4)

	#find the triggers in offline data
	def findTriggersOffline(self):
		#use a rolling window to find the triggers that should pass
		self.myarray[offlineTrigNames] = self.myarray.rolling(window='{}s'.format(trigWindow), on="fake_time", axis=0).agg({"fourLayers": self.this4Layers, "threeInRow": self.checkThreeInRow, "separateLayers": self.checkSeparate, "adjacentLayers": self.checkAdjacent, 'nLayers': self.thisNLayers, 'nHits': self.thisNPulses, 'topPanels': self.checkTopPanels, 'topBotPanels': self.topBotPanels, 'frontBack': self.checkThroughGoing})

	#convert all triggers to binary
	def convertTriggers(self, df):
		trig = format(int(df), '#016b').split("b")[-1]
		trig = trig[::-1]
		return trig

	#get value of specific trigger bit
	def getTriggerBit(self, df, bit):
		return int(df[bit])

	#get triggers passed online
	def getOnlineTriggers(self):
		self.myarray['triggerString'] = self.myarray.apply(lambda row: self.convertTriggers(row.tTrigger), axis=1)
		for i in range(13):
			self.myarray['trig{}'.format(i)] = self.myarray.apply(lambda row: self.getTriggerBit(row.triggerString, i), axis=1)

	#open the output file for plots
	def openOutputFile(self):
		outputFilename = os.getcwd() + '/triggerRates_Run{0}.{1}.root'.format(self.runNum, self.fileNum)
		self.file_out = r.TFile.Open(outputFilename, 'recreate')

In [3]:
r.gROOT.SetBatch(1)
#filename2 = '/store/user/milliqan/trees/v31/MilliQan_Run1031.2_v31_firstPedestals.root'
filename1 = '/store/user/milliqan/trees/v33/MilliQan_Run1118.1_v33_firstPedestals.root'
mychecker = triggerChecker()
#mychecker.openFile(filename2)
#data1 = mychecker.myarray
mychecker.openFile(filename1)
#data2 = mychecker.myarray


In [4]:
array=mychecker.myarray

In [9]:
array["area"].max()

646678.5625

In [28]:
array["DAQEventNumber"]==1

entry  subentry
0      0            True
       1            True
       2            True
1      0           False
       1           False
                   ...  
998    0           False
999    0           False
       1           False
       2           False
       3           False
Name: DAQEventNumber, Length: 3049, dtype: bool

In [29]:
chanList=array["chan"]

In [30]:
chanset =set(chanList)

In [31]:
chanset

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 65,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 78,
 79}

In [33]:
for number in range(3):
    for chan in chanset:
        if chan > 71 and chan < 75: 
            print(number)
            print(chan)
            break
    print("reach here")

0
72
reach here
1
72
reach here
2
72
reach here


In [15]:
array["event"].max() + 1

1000

In [6]:
array["DAQEventNumber"].max()

1000

In [ ]:
data1[0:230]

In [6]:
data1[data1["event"]==1]

event  fileNumber    time        nPE      height  \
entry subentry                                                     
1     0             1           2  1300.0   1.979721   73.480957   
      1             1           2   282.5   0.232555   21.872108   
      2             1           2  1212.5   0.847139   30.390627   
      3             1           2   407.5   0.420888   24.104057   
      4             1           2   580.0   1.737588   72.596352   
      5             1           2  1242.5  20.052887  610.310181   
      6             1           2  2230.0   0.510205   16.553974   

                        area  chan  row  column  layer  type  \
entry subentry                                                 
1     0          2474.650879     0    3       0      0     0   
      1           255.810455     2    2       0      0     0   
      2           847.138916    16    3       0      1     0   
      3           631.332397    32    3       0      2     0   
      4          2606.381348    32    3       0      2     0   
      5         30079.330078    32    3       0      2     0   
      6           765.306946    32    3       0      2     0   

                event_time_fromTDC  tTrigger   duration  npulses  \
entry subentry                                                     
1     0               1.686213e+09      -1.0  162.48999        1   
      1               1.686213e+09      -1.0   44.98999        1   
      2               1.686213e+09      -1.0  107.48999        1   
      3               1.686213e+09      -1.0   94.98999        4   
      4               1.686213e+09      -1.0  172.48999        4   
      5               1.686213e+09      -1.0  657.48999        4   
      6               1.686213e+09      -1.0  149.98999        4   

                DAQEventNumber  pickupFlag  board  
entry subentry                                     
1     0                   1002       False      0  
      1                   1002        True      0  
      2                   1002       False      1  
      3                   1002       False      2  
      4                   1002       False      2  
      5                   1002       False      2  
      6                   1002       False      2

In [17]:
data2[0:230]

event  fileNumber    time         nPE       height  \
entry subentry                                                       
0     0             0           1  1160.0    0.477263    22.094797   
      1             0           1    55.0    0.044608    10.052223   
      2             0           1  1212.5    2.123607    37.336296   
      3             0           1   930.0   89.718307  1214.605591   
      4             0           1  1290.0    1.604758    38.878670   
...               ...         ...     ...         ...          ...   
15    10           15           1  2240.0  615.707764    25.711159   
16    0            16           1  1322.5   63.160561  1250.066772   
      1            16           1  1325.0   12.491386   303.741211   
      2            16           1  2140.0    0.241976    16.433935   
      3            16           1  1312.5   63.133438  1249.261597   

                        area  chan  row  column  layer  type  \
entry subentry                                                 
0     0           596.578308     0    3       0      0     0   
      1            44.608055    16    3       0      1     0   
      2          1486.524780    33    3       1      2     0   
      3         53830.984375    48    3       0      3     0   
      4           962.854614    48    3       0      3     0   
...                      ...   ...  ...     ...    ...   ...   
15    10          615.707764    69    0       4      0     2   
16    0         78950.703125     0    3       0      0     0   
      1         13740.524414     2    2       0      0     0   
      2           266.173431     2    2       0      0     0   
      3         63133.437500    15    0       3      0     0   

                event_time_fromTDC  tTrigger    duration  npulses  \
entry subentry                                                      
0     0               1.686213e+09      -1.0   97.489990        1   
      1               1.686213e+09      -1.0   12.489998        1   
      2               1.686213e+09      -1.0  154.989990        1   
      3               1.686213e+09      -1.0  319.989990        2   
      4               1.686213e+09      -1.0  102.489990        2   
...                            ...       ...         ...      ...   
15    10              1.686213e+09      -1.0   87.489990        2   
16    0               1.686213e+09      -1.0  339.989990        1   
      1               1.686213e+09      -1.0  327.489990        2   
      2               1.686213e+09      -1.0   52.489990        2   
      3               1.686213e+09      -1.0  294.989990        2   

                DAQEventNumber  pickupFlag  
entry subentry                              
0     0                      1       False  
      1                      1        True  
      2                      1       False  
      3                      1       False  
      4                      1       False  
...                        ...         ...  
15    10                    16       False  
16    0                     17       False  
      1                     17       False  
      2                     17       False  
      3                     17       False  

[230 rows x 17 columns]

In [14]:
data2[(data2["event"] < 10) & (data2["event"] > 1)]

event  fileNumber    time           nPE       height  \
entry subentry                                                         
2     0             2           1    50.0     10.582748   139.754852   
      1             2           1   427.5     38.654640   419.977081   
      2             2           1  1045.0     13.780471   394.280243   
      3             2           1  1300.0      3.515676   101.929115   
      4             2           1  1602.5      5.320735   163.117706   
...               ...         ...     ...           ...          ...   
9     10            9           1    50.0    476.306458    31.156055   
      11            9           1  1225.0   3931.357910    79.337288   
      12            9           1  1315.0    593.700867    27.700769   
      13            9           1  1125.0   4512.475098    61.609375   
      14            9           1  1112.5  80373.476562  1246.259033   

                        area  chan  row  column  layer  type  \
entry subentry                                                 
2     0          8466.198242    31    0       3      1     0   
      1         30923.712891    31    0       3      1     0   
      2         11024.376953    31    0       3      1     0   
      3          2812.541260    31    0       3      1     0   
      4          4256.588379    31    0       3      1     0   
...                      ...   ...  ...     ...    ...   ...   
9     10          476.306488    27    0       1      1     0   
      11         3931.357910    27    0       1      1     0   
      12          593.700928    38    2       2      2     0   
      13         4512.475098    57    1       1      3     0   
      14        80373.476562    73    0       4      2     2   

                event_time_fromTDC  tTrigger    duration  npulses  \
entry subentry                                                      
2     0               1.686213e+09      -1.0  309.989990        7   
      1               1.686213e+09      -1.0  514.989990        7   
      2               1.686213e+09      -1.0  162.489990        7   
      3               1.686213e+09      -1.0  117.489990        7   
      4               1.686213e+09      -1.0  134.989990        7   
...                            ...       ...         ...      ...   
9     10              1.686213e+09      -1.0   69.989998        2   
      11              1.686213e+09      -1.0  222.489990        2   
      12              1.686213e+09      -1.0   77.489990        1   
      13              1.686213e+09      -1.0  304.989990        1   
      14              1.686213e+09      -1.0  359.989990        1   

                DAQEventNumber  pickupFlag  board  
entry subentry                                     
2     0                      3        True      1  
      1                      3       False      1  
      2                      3       False      1  
      3                      3       False      1  
      4                      3       False      1  
...                        ...         ...    ...  
9     10                    10        True      1  
      11                    10       False      1  
      12                    10       False      2  
      13                    10       False      3  
      14                    10       False      4  

[86 rows x 18 columns]

In [19]:
result = pd.concat([data1, data2], ignore_index=True)

In [22]:
result[990:1003]

,event,fileNumber,time,nPE,height,area,chan,row,column,layer,type,event_time_fromTDC,tTrigger,duration,npulses,DAQEventNumber,pickupFlag
990,73,2,2347.5,74.504890,11.597141,74.504890,55,2,3,3,0,1.686213e+09,-1.0,19.98999,7,1074,True
991,73,2,2395.0,44.179478,10.110500,44.179478,55,2,3,3,0,1.686213e+09,-1.0,12.48999,7,1074,True
992,73,2,2452.5,48.890831,10.219769,48.890831,55,2,3,3,0,1.686213e+09,-1.0,12.48999,7,1074,True
993,73,2,2477.5,107.620155,11.214077,107.620155,55,2,3,3,0,1.686213e+09,-1.0,27.48999,7,1074,True
994,73,2,1340.0,844.929932,22.338814,844.929932,69,0,4,0,2,1.686213e+09,-1.0,134.98999,1,1074,False
995,73,2,1292.5,4608.877441,107.419228,4608.877441,70,0,-1,0,2,1.686213e+09,-1.0,192.48999,1,1074,False
996,74,2,1312.5,203.303940,1252.063721,142312.765625,51,2,1,3,0,1.686213e+09,-1.0,469.98999,2,1075,False
997,74,2,2457.5,10.979869,271.831757,7685.908691,51,2,1,3,0,1.686213e+09,-1.0,99.98999,2,1075,False
998,74,2,1312.5,4668.666504,182.040741,4668.666504,54,2,2,3,0,1.686213e+09,-1.0,167.48999,1,1075,False
999,74,2,1450.0,169.485779,19.017254,169.485779,58,0,0,3,0,1.686213e+09,-1.0,37.48999,1,1075,True


In [28]:
result

,event,fileNumber,time,nPE,height,area,chan,row,column,layer,type,event_time_fromTDC,tTrigger,duration,npulses,DAQEventNumber,pickupFlag
0,0,2,80.0,1210.068359,54.601185,1210.068359,23,2,3,1,0,1.686213e+09,-1.0,104.990005,2,1001,True
1,0,2,1217.5,5161.270996,213.573837,5161.270996,23,2,3,1,0,1.686213e+09,-1.0,164.989990,2,1001,False
2,0,2,595.0,1.149382,34.049633,919.505798,31,0,3,1,0,1.686213e+09,-1.0,107.489990,1,1001,False
3,0,2,1300.0,10467.139648,271.375916,10467.139648,56,1,0,3,0,1.686213e+09,-1.0,239.989990,1,1001,False
4,1,2,1300.0,1.979721,73.480957,2474.650879,0,3,0,0,0,1.686213e+09,-1.0,162.489990,1,1002,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26675,999,1,265.0,1068.069702,34.525829,1068.069702,42,0,0,2,0,1.686213e+09,-1.0,119.989990,3,1000,False
26676,999,1,1267.5,1958.927612,54.617390,1958.927612,42,0,0,2,0,1.686213e+09,-1.0,154.989990,3,1000,False
26677,999,1,1745.0,1913.688843,49.891663,1913.688965,42,0,0,2,0,1.686213e+09,-1.0,169.989990,3,1000,False
26678,999,1,1130.0,2402.614502,70.214119,2402.614502,59,0,1,3,0,1.686213e+09,-1.0,157.489990,1,1000,False


In [10]:
data["npulses"].max()

22

In [9]:
data["npulses"].min()

NameError: name 'data' is not defined

In [13]:
data2[data2["npulses"]<5]

event  fileNumber    time          nPE       height  \
entry subentry                                                        
0     0             0           1  1160.0     0.477263    22.094797   
      1             0           1    55.0     0.044608    10.052223   
      2             0           1  1212.5     2.123607    37.336296   
      3             0           1   930.0    89.718307  1214.605591   
      4             0           1  1290.0     1.604758    38.878670   
...               ...         ...     ...          ...          ...   
999   0           999           1   265.0  1068.069702    34.525829   
      1           999           1  1267.5  1958.927612    54.617390   
      2           999           1  1745.0  1913.688843    49.891663   
      3           999           1  1130.0  2402.614502    70.214119   
      4           999           1  2105.0     0.102282    18.827579   

                        area  chan  row  column  layer  type  \
entry subentry                                                 
0     0           596.578308     0    3       0      0     0   
      1            44.608055    16    3       0      1     0   
      2          1486.524780    33    3       1      2     0   
      3         53830.984375    48    3       0      3     0   
      4           962.854614    48    3       0      3     0   
...                      ...   ...  ...     ...    ...   ...   
999   0          1068.069702    42    0       0      2     0   
      1          1958.927612    42    0       0      2     0   
      2          1913.688965    42    0       0      2     0   
      3          2402.614502    59    0       1      3     0   
      4            92.053734    63    0       3      3     0   

                event_time_fromTDC  tTrigger    duration  npulses  \
entry subentry                                                      
0     0               1.686213e+09      -1.0   97.489990        1   
      1               1.686213e+09      -1.0   12.489998        1   
      2               1.686213e+09      -1.0  154.989990        1   
      3               1.686213e+09      -1.0  319.989990        2   
      4               1.686213e+09      -1.0  102.489990        2   
...                            ...       ...         ...      ...   
999   0               1.686213e+09      -1.0  119.989990        3   
      1               1.686213e+09      -1.0  154.989990        3   
      2               1.686213e+09      -1.0  169.989990        3   
      3               1.686213e+09      -1.0  157.489990        1   
      4               1.686213e+09      -1.0   22.489990        1   

                DAQEventNumber  pickupFlag  board  
entry subentry                                     
0     0                      1       False      0  
      1                      1        True      1  
      2                      1       False      2  
      3                      1       False      3  
      4                      1       False      3  
...                        ...         ...    ...  
999   0                   1000       False      2  
      1                   1000       False      2  
      2                   1000       False      2  
      3                   1000       False      3  
      4                   1000        True      3  

[9490 rows x 18 columns]

One event per entry

In [5]:
data.index

MultiIndex([(  0,  1),
            (  0,  9),
            (  0,  5),
            (  0,  3),
            (  0,  0),
            (  0,  2),
            (  0,  4),
            (  0,  6),
            (  0,  7),
            (  0, 10),
            ...
            (998,  8),
            (998, 10),
            (998, 13),
            (998, 11),
            (998,  6),
            (999,  0),
            (999,  3),
            (999,  1),
            (999,  2),
            (999,  4)],
           names=['entry', 'subentry'], length=13157)

In [6]:
data["height"] > 100

entry  subentry
0      1           False
       9           False
       5            True
       3            True
       0           False
                   ...  
999    0           False
       3           False
       1           False
       2           False
       4           False
Name: height, Length: 13157, dtype: bool

In [7]:
data[data["height"] > 100]["area"]

entry  subentry
0      5            84523.656250
       3            53830.984375
1      5            41676.609375
       0            21895.910156
       12           58573.175781
                       ...      
998    7           105605.210938
       9            80779.109375
       12          132910.812500
       5             7033.617188
       6             1913.588501
Name: area, Length: 4669, dtype: float32

In [8]:
#unpack?
for area in data[data["height"] > 100]["area"]:
    print(area)

84523.65625
53830.984375
41676.609375
21895.91015625
58573.17578125
8466.1982421875
30923.712890625
45355.34765625
80173.8984375
6016.9462890625
17532.208984375
11024.376953125
2812.541259765625
4256.58837890625
13025.2294921875
44612.70703125
113516.21875
6479.84912109375
10195.24609375
2154.189453125
5413.41943359375
6824.927734375
29443.9765625
20701.884765625
6461.375
57749.6015625
6134.10546875
28935.685546875
21235.576171875
15506.8935546875
14622.8828125
26932.787109375
12229.263671875
6161.2353515625
80373.4765625
497236.625
540126.4375
15992.38671875
5511.8486328125
48642.3046875
45294.1953125
83583.3515625
23323.044921875
19795.83984375
77561.625
25847.181640625
79648.4296875
46454.8828125
15938.875
50190.8984375
89342.375
10473.3671875
237796.765625
97041.6796875
3140.24365234375
26143.5625
12821.728515625
10498.7294921875
3875.8349609375
13847.2998046875
5449.2958984375
11663.82421875
30264.5703125
51409.8671875
88903.0
23646.41796875
46764.7890625
8874.4482421875
5212.0146

16231.7958984375
6426.61474609375
7672.61328125
39062.75
11568.171875
33357.87109375
85559.3515625
11552.8037109375
26142.4140625
62892.953125
6921.49560546875
96477.3984375
65871.203125
7128.38427734375
9505.0107421875
10855.1806640625
2833.2412109375
11909.9189453125
28836.431640625
6224.9482421875
11895.3271484375
9807.1845703125
38093.046875
10060.5205078125
4418.61767578125
2980.6083984375
197043.34375
3698.43701171875
5004.716796875
5394.18408203125
57119.796875
66385.4140625
8910.0595703125
82149.109375
13904.55078125
70064.5546875
24584.525390625
33959.8125
10439.7490234375
13941.626953125
13453.5693359375
28097.0078125
22002.607421875
11462.208984375
6765.44580078125
6160.212890625
7035.48193359375
7974.12060546875
6559.91357421875
20395.48828125
73516.390625
19612.5234375
50804.09375
62031.55078125
9234.142578125
13965.1650390625
16081.142578125
3691.398193359375
32714.28125
13595.611328125
2148.435546875
4644.93017578125
12622.658203125
34983.83203125
78874.75
6707.099121093

3016.852294921875
95781.921875
155564.484375
520918.125
78481.5546875
56217.73828125
14410.8037109375
64131.6796875
124807.9921875
53129.56640625
33423.078125
79743.6875
110215.5546875
98098.9921875
5851.93017578125
30872.5
4668.16650390625
5249.673828125
11402.5078125
4799.71044921875
1765.7008056640625
6794.1708984375
4437.70556640625
75547.921875
24976.291015625
10788.55078125
12651.107421875
56100.96484375
33749.921875
5921.7646484375
86925.6875
9259.0810546875
9930.505859375
5961.15087890625
4041.58984375
12277.7158203125
3730.32421875
24784.0703125
104402.5078125
12625.16796875
21632.44921875
58150.3125
11400.0927734375
75919.8125
56549.7578125
4096.84619140625
54559.3515625
41260.71484375
16312.4541015625
24672.455078125
3705.93505859375
3337.325439453125
25558.771484375
80113.5078125
62716.46875
11411.3935546875
16810.578125
4804.43701171875
11346.6181640625
12679.8515625
11831.0595703125
4550.4267578125
48709.625
47546.6015625
36641.11328125
5938.4462890625
40554.68359375
2133

50797.06640625
48791.109375
15162.712890625
8055.01806640625
42608.57421875
32836.00390625
37714.70703125
25836.142578125
9450.5380859375
8140.962890625
6165.8046875
83616.5703125
49191.3515625
5368.6875
70510.640625
70739.4765625
54822.55078125
52034.24609375
90673.3515625
42502.2109375
2561.676025390625
4780.0400390625
25317.70703125
13284.8056640625
41694.87890625
56449.71484375
12972.82421875
8238.048828125
13292.1455078125
4829.34228515625
4366.21875
11591.2998046875
5522.59765625
81674.8515625
82031.6953125
70269.4765625
7864.31396484375
21824.00390625
14409.5029296875
2852.15185546875
58811.96875
16053.4580078125
7317.087890625
5387.28759765625
50407.9140625
27655.763671875
23439.330078125
61655.87109375
63534.3671875
43275.25390625
5002.4541015625
11365.26953125
21496.6328125
105872.5546875
14777.154296875
3662.86962890625
654414.5
179389.796875
43913.015625
94403.671875
15107.626953125
494123.84375
629167.0
625295.125
4678.11083984375
21794.5546875
178085.78125
50306.54296875


6455.677734375
3324.39208984375
8192.0068359375
4109.58251953125
4949.34912109375
13681.5390625
7126.09912109375
12081.291015625
128570.828125
62210.95703125
101262.8984375
7571.658203125
4784.77880859375
7087.017578125
29309.041015625
8319.794921875
95412.3671875
9373.0966796875
10793.89453125
43131.37890625
26160.89453125
10153.607421875
44794.41015625
31063.07421875
11420.6005859375
4523.01513671875
85828.421875
3631.62646484375
2644.908935546875
49973.8828125
136700.6875
92693.6484375
75871.2734375
7705.89111328125
5327.01416015625
5796.978515625
78068.2265625
6429.52294921875
12317.1640625
76126.2890625
16590.064453125
23339.740234375
93305.234375
6024.4443359375
2703.690185546875
24525.814453125
15416.2392578125
82326.234375
26321.181640625
20911.396484375
77161.7265625
1265.6456298828125
260653.609375
56780.82421875
16336.1904296875
14258.67578125
72780.375
81641.2421875
16322.46484375
11568.9951171875
4573.95068359375
11215.4248046875
3427.20703125
2210.490478515625
16684.30859

In [19]:
#count the number pulse above 100 in event 1
pulse_count = data[data["height"] > 100]["event"]==0 #return true false table
num = pulse_count.sum()
print(num)

2


In [22]:
eventList=set(data["event"])

print(len(eventList))

1000


In [25]:
data["row"]

entry  subentry
0      1           3
       9          -1
       5           2
       3           3
       0           3
                  ..
999    0           0
       3           0
       1           0
       2           0
       4           0
Name: row, Length: 13157, dtype: int32

In [7]:
a = []
b = [a]
b[0].append("1")
b

[['1']]

In [9]:
c = [a]
c

[['1']]